In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path

In [2]:
# auxiliary function for below
def split(theList, n):
    for i in range(0, len(theList), n):
        yield theList[i:i + n]

In [3]:
host = "https://api.stratz.com/api/v1/match"

In [4]:
params = {"matchId":[3304258209,3304204784], 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}

In [5]:
x = requests.get(host, params = params)

In [6]:
x

<Response [200]>

In [7]:
host = "https://api.stratz.com/api/v1/match/3304258209"
rs = (grequests.get(host),)
x = grequests.map(rs)
data = x[0].json()

In [8]:
keys = data.keys()
print(keys)

dict_keys(['clusterId', 'challenge', 'id', 'lobbyType', 'rank', 'roshanEvents', 'gameVersionId', 'runeEvents', 'isStats', 'gameResult', 'firstBloodTime', 'didRadiantWin', 'duration', 'gameMode', 'parsedDate', 'wardEvents', 'towerDeathEvents', 'avgImp', 'bracket', 'regionId', 'direKills', 'players', 'buildingEvents', 'radiantKills', 'replaySalt', 'tier', 'courierEvents', 'endDate'])


In [9]:
testsequence = [event['item'] for event in data['players'][0]['purchaseEvents']]
testitems = [45, 42, 44]
list(filter(lambda a: a not in testitems, testsequence))

[188,
 43,
 29,
 38,
 46,
 46,
 84,
 244,
 27,
 27,
 214,
 46,
 46,
 46,
 25,
 182,
 46,
 46,
 64,
 65,
 46,
 46,
 23,
 21,
 22,
 60,
 108]

In [10]:
keys

dict_keys(['clusterId', 'challenge', 'id', 'lobbyType', 'rank', 'roshanEvents', 'gameVersionId', 'runeEvents', 'isStats', 'gameResult', 'firstBloodTime', 'didRadiantWin', 'duration', 'gameMode', 'parsedDate', 'wardEvents', 'towerDeathEvents', 'avgImp', 'bracket', 'regionId', 'direKills', 'players', 'buildingEvents', 'radiantKills', 'replaySalt', 'tier', 'courierEvents', 'endDate'])

In [11]:
data['players'][0].keys()

dict_keys(['role', 'abilityLearnEvents', 'item3', 'heroDamage', 'level', 'abilityActiveLists', 'item4', 'item1', 'playerUpdateLevelEvents', 'item0', 'goldPerMinute', 'award', 'expPerMinute', 'healEvents', 'playerUpdateGoldEvents', 'soloRank', 'isRadiant', 'name', 'deathEvents', 'leaverStatus', 'lane', 'imp', 'playerUpdatePositionEvents', 'towerDamageEvents', 'inventoryEvent', 'assistEvents', 'heroDamageEvents', 'purchaseEvents', 'avatar', 'steamId', 'numDeaths', 'avatarMedium', 'numDenies', 'slot', 'killEvents', 'goldSpent', 'heroAvgImp', 'heroHealing', 'csEvents', 'avatarFull', 'abilityUsedEvents', 'item5', 'playerUpdateAttributeEvents', 'buyBackEvents', 'towerDamage', 'experienceEvents', 'numAssists', 'goldEvents', 'partyRank', 'hero', 'playerUpdateHealthEvents', 'profileUrl', 'numKills', 'playerUpdateBattleEvents', 'item2', 'numLastHits'])

In [12]:
print(data['players'][0]['role'])
print(data['players'][0]['lane'])
print(data['players'][0]['hero'])
print(data['players'][0]['purchaseEvents'])
for player in data['players']:
    print(player['slot'])

1
1
92
[{'item': 188, 'time': -89}, {'item': 43, 'time': -89}, {'item': 29, 'time': -89}, {'item': 44, 'time': -89}, {'item': 38, 'time': -89}, {'item': 46, 'time': -85}, {'item': 46, 'time': -84}, {'item': 84, 'time': 187}, {'item': 42, 'time': 193}, {'item': 42, 'time': 236}, {'item': 42, 'time': 384}, {'item': 244, 'time': 389}, {'item': 27, 'time': 389}, {'item': 27, 'time': 432}, {'item': 214, 'time': 440}, {'item': 46, 'time': 509}, {'item': 42, 'time': 528}, {'item': 46, 'time': 603}, {'item': 46, 'time': 719}, {'item': 42, 'time': 720}, {'item': 25, 'time': 809}, {'item': 182, 'time': 811}, {'item': 42, 'time': 816}, {'item': 46, 'time': 882}, {'item': 46, 'time': 1059}, {'item': 64, 'time': 1264}, {'item': 65, 'time': 1264}, {'item': 46, 'time': 1277}, {'item': 46, 'time': 1533}, {'item': 23, 'time': 1758}, {'item': 21, 'time': 1758}, {'item': 22, 'time': 1758}, {'item': 60, 'time': 1760}, {'item': 108, 'time': 1760}, {'item': 42, 'time': 1919}]
0
1
2
3
4
128
129
130
131
132


**Need to get steamIDs for players by going through the matchIDs of high level games**

In [13]:
def nextBatchOfMatches(idList):
    queryParams = {"matchId":idList, 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        for player in entry['players']:
            playerSet.add(player['steamId'])

In [14]:
if os.path.exists('playerset.txt'):
    playerSet = np.loadtxt('playerset.txt', dtype=np.int64)
else:
    playerSet = set()
    allMatches = np.loadtxt('matchids.txt', dtype=np.int64)
    allMatches = split(allMatches, 10)
    for matches in allMatches:
        nextBatchOfMatches(matches)
    playerSet = list(playerSet)
    np.savetxt('playerset.txt', playerSet, delimiter=',', fmt = "%.0f")

With player set, we can obtain matches. For the players above, get the matches these players play and also categorize the players and heroes

In [15]:
# 8 is jug and 74 is invoker and 106 is ember
def playerMatchObtain(player, heroList = [8, 74, 106]):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": player, 
                   "heroId": heroList,
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        matchSet.add(entry['id'])

In [16]:
matchSet = set()
if os.path.exists('matchset.txt'):
    matchSet = np.loadtxt('matchset.txt', dtype=np.int64)
else:
    allplayers = np.loadtxt('playerset.txt', dtype=np.int64)
    for player in allplayers:
        playerMatchObtain(player, heroList=[74])
    matchSet = list(matchSet)
    np.savetxt('matchset.txt', matchSet, delimiter=',', fmt = "%.0f")

In [17]:
len(playerSet)

589

In [18]:
len(matchSet)

1542

** Time to Parse these Matches**
Need to make sure to remove tps and wards from buy list as you can buy them at any time depending on the situation

In [58]:
prototypeDataframe = {
    "invokerItems": [], "invokerSideRadiant": [], 
    "invokerSideHero1": [], "invokerSideHero2": [], 
    "invokerSideHero3": [], "invokerSideHero4": [],
    "otherSideHero1": [], "otherSideHero2": [],
    "otherSideHero3": [], "otherSideHero4": [],
    "otherSideHero5": [], 
    "invokerSideHero1Steam": [], "invokerSideHero2Steam": [], 
    "invokerSideHero3Steam": [], "invokerSideHero4Steam": [],
    "otherSideHero1Steam": [], "otherSideHero2Steam": [],
    "otherSideHero3Steam": [], "otherSideHero4Steam": [],
    "otherSideHero5Steam": [], "matchId": []
}
session = requests.Session()
def analyzeMatchesForFeatures(matchList, heroesToTestItems = [8, 74, 106], ):
    hostInitial = "https://api.stratz.com/api/v1/match/"
    rs = (grequests.get(hostInitial + str(match), session=session) for match in matchList)
    x = grequests.map(rs, size = 50)
    for matchUnparsed in x:
        if not matchUnparsed.ok:
            matchUnparsed.close()
            continue
        match = matchUnparsed.json()
        radiantHeroes = []
        direHeroes = []
        radiantSteams = []
        direSteams = []
        invokerSideRadiant = None
        for player in match['players']:
            # we have the invoker
            if player['hero'] == 74:
                invokerSideRadiant = player['slot'] < 5
                invokerItems = [event['item'] for event in player['purchaseEvents']]
            else:
                if player['slot'] < 5:
                    radiantHeroes.append(player['hero'])
                    radiantSteams.append(player['steamId'])
                else: 
                    direHeroes.append(player['hero'])
                    direSteams.append(player['steamId'])
        invokerWon = invokerSideRadiant == match['didRadiantWin']
        if not invokerWon:
            matchUnparsed.close()
            continue
        prototypeDataframe['matchId'].append(match['id'])
        prototypeDataframe['invokerSideRadiant'].append(invokerSideRadiant)
        prototypeDataframe['invokerItems'].append(invokerItems)
        if invokerSideRadiant:
            invokerSideHeroes = radiantHeroes
            invokerSideSteams = radiantSteams
            otherSideHeroes = direHeroes
            otherSideSteams = direSteams
        else:
            invokerSideHeroes = direHeroes
            invokerSideSteams = direSteams
            otherSideHeroes = radiantHeroes
            otherSideSteams = radiantSteams
        for ix1 in range(5):
            prototypeDataframe['otherSideHero' + str(ix1+1)].append(otherSideHeroes[ix1])
            prototypeDataframe['otherSideHero' + str(ix1+1) + 'Steam'].append(otherSideSteams[ix1])
            if ix1 != 4:
                prototypeDataframe['invokerSideHero' + str(ix1+1)].append(invokerSideHeroes[ix1])
                prototypeDataframe['invokerSideHero'+ str(ix1+1) + 'Steam'].append(invokerSideSteams[ix1])
        matchUnparsed.close()
            
def GetHeroHistory(steamid, heroid):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": steamid, 
                   "heroId": heroid,
                   "include": "Player",
                   "playerType": "Single",
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = queryParams)
    print(x)
    x = x.json()
    features = {'kills': [], 'deaths': [], 'lastHits': [], 'gold': [], 'obs': [], 'sens': []}
    print(x['results'][0].keys())
    print(x['results'][0]['players'][0].keys())

In [59]:
GetHeroHistory(54325937, 74)

<Response [200]>
dict_keys(['clusterId', 'firstBloodTime', 'didRadiantWin', 'duration', 'gameMode', 'parsedDate', 'id', 'direKills', 'avgImp', 'bracket', 'rank', 'endDate', 'players', 'isStats', 'radiantKills', 'gameResult', 'tier', 'replaySalt', 'lobbyType'])
dict_keys(['role', 'level', 'partyId', 'item3', 'award', 'avatar', 'steamId', 'numDeaths', 'heroDamage', 'item4', 'numDenies', 'slot', 'name', 'heroHealing', 'item1', 'avatarMedium', 'goldSpent', 'towerDamage', 'item5', 'avatarFull', 'lane', 'item0', 'isRadiant', 'partyRank', 'expPerMinute', 'item2', 'soloRank', 'goldPerMinute', 'hero', 'imp', 'leaverStatus', 'profileUrl', 'numKills', 'numAssists', 'numLastHits'])


In [ ]:
if not os.path.exists('invokerMatchFeatureSetPlus.csv'):
    #analyzeMatchesForFeatures(matchSet[0:20])
    #invokerMatchFeatureSetPlus = pd.DataFrame(prototypeDataframe)
    #invokerMatchFeatureSetPlus.to_csv('invokerMatchFeatureSetPlus.csv', index=False)